In [1]:
from LSTM import *


2024-04-13 23:18:09.797007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_data = pd.read_csv('data/SPY_data_5min.csv')
input_data['time'] = pd.to_datetime(input_data['time'])

# Filter trades within the time window from 09:30 to 16:00
input_data = input_data[(input_data['time'].dt.time >= pd.to_datetime('09:30').time()) & 
                 (input_data['time'].dt.time <= pd.to_datetime('16:00').time())]

input_data.drop(['Unnamed: 0','money','open','high','low'], axis=1, inplace=True)
input_data['time'] = pd.to_datetime(input_data['time']).dt.strftime('%H:%M')


In [3]:
lstm = DataPreparationLSTM(
    df = input_data,
    future = 20,
    lag = 20,
    min_max_scaler=True,
    log_transform=True,
    semi_variance=True,
    jump_detect=True,
    period_train= list(
                    [
                        pd.to_datetime("2004-01-02",format = "%Y-%m-%d"),
                        pd.to_datetime("2010-12-31",format = "%Y-%m-%d"),
                    ]
                ),
                period_test= list(
                    [
                        pd.to_datetime("2011-01-01",format = "%Y-%m-%d"),
                        pd.to_datetime("2013-12-31",format = "%Y-%m-%d"),
                    ]
                ),
)

lstm.prepare_all()

tf.keras.backend.clear_session()

model = LSTM(
    lstm.training_set,
    lstm.testing_set,
    lstm.train_matrix,
    lstm.train_y,
    lstm.test_matrix,
    lstm.test_y,
    epochs = 70,
    learning_rate = 0.01,
    layer_one = 40,
    layer_two = 40,
    layer_three = 0,
    layer_four = 0
)

model.make_accuracy_measures()

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/LSTM.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)
2024-04-13 23:18:15.346414: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/70
53/53 [==============================] - 5s 50ms/step - loss: 0.0214 - val_loss: 0.0053
Epoch 2/70
53/53 [==============================] - 2s 41ms/step - loss: 0.0055 - val_loss: 0.0034
Epoch 3/70
53/53 [==============================] - 2s 39ms/step - loss: 0.0048 - val_loss: 0.0032
Epoch 4/70
53/53 [==============================] - 2s 31ms/step - loss: 0.0048 - val_loss: 0.0030
Epoch 5/70
53/53 [==============================] - 1s 26ms/step - loss: 0.0048 - val_loss: 0.0031
Epoch 6/70
53/53 [==============================] - 1s 28ms/step - loss: 0.0043 - val_loss: 0.0029
Epoch 7/70
53/53 [==============================] - 1s 27ms/step - loss: 0.0045 - val_loss: 0.0038
Epoch 8/70
53/53 [==============================] - 2s 31ms/step - loss: 0.0041 - val_loss: 0.0029
Epoch 9/70
53/53 [==============================] - 2s 37ms/step - loss: 0.0039 - val_loss: 0.0027
Epoch 10/70
53/53 [==============================] - 2s 35ms/step - loss: 0.0040 - val_loss: 0.0027
Epoch 11/

In [4]:
model.test_accuracy

{'MSE': 0.005886503905786027,
 'MAE': 0.059187877420616784,
 'RSquared': 0.5550306297478863}

In [5]:
model.train_accuracy

{'MSE': 0.007725745768244295,
 'MAE': 0.06556761180901355,
 'RSquared': 0.7131965292565485}

In [6]:
model.fitness

1.268227159004435